In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

In [ ]:
with open("tokenizer.pkl", "rb") as f:
    stoi, itos = pickle.load(f)

def encode(s): return [stoi.get(c, stoi[' ']) for c in s]
def decode(l): return ''.join([itos[i] for i in l])

vocab_size = len(stoi)


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
batch_size = 64
block_size = 256
n_embed = 256
n_heads = 8
n_layer = 6
learning_rate = 1e-4
max_iters = 3000
eval_interval = 100

In [5]:
# Attention class

class SelfAttentionHead(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.query = nn.Linear(n_embed,head_size,bias = False)
        self.key = nn.Linear(n_embed,head_size,bias = False)
        self.value = nn.Linear(n_embed,head_size,bias = False)
        self.dropout = nn.Dropout(0.1)
        self.register_buffer("tril",torch.tril(torch.ones(block_size,block_size)))

    def forward(self,x):
        B,T,C = x.shape
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        wei = q @ k.transpose(-2,-1) * (C**-0.5)
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
        wei = F.softmax(wei,dim=-1)
        wei = self.dropout(wei)

        return wei @ v

In [6]:
# Transformers class

class TransformerBlock(nn.Module):
    def __init__(self,n_embed,n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa_heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embed,n_embed)
        self.ffw = nn.Sequential(
            nn.Linear(n_embed,n_embed),
            nn.ReLU(),
            nn.Linear(n_embed,n_embed)
        )
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        self.dropout = nn.Dropout(0.1)


    def forward(self,x):
        x = x + self.dropout(self.proj(torch.cat([h(x) for h in self.sa_heads], dim=-1))) # resnet connection 1
        x = self.ln1(x)
        x = x + self.ffw(x) # resnet connection 2
        x = self.ln2(x)

        return x

In [ ]:
# Language Model class
class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size,n_embed)
        self.pos_embed = nn.Embedding(block_size,n_embed)
        self.blocks = nn.Sequential(*[TransformerBlock(n_embed,n_heads) for _ in range(n_layer)])
        self.ln = nn.LayerNorm(n_embed)
        self.lm = nn.Linear(n_embed,vocab_size)

    def forward(self, idx,target=None):

        B,T = idx.shape
        static_embed = self.token_embed(idx)
        positional_embed  = self.pos_embed(torch.arange(T, device=idx.device))
        x = static_embed + positional_embed
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.lm(x)

        if target is None:
            return logits , None

        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        target = target.view(B*T)
        loss = F.cross_entropy(logits,target)

        return logits,loss


    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None, stop_token=None):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float('Inf')

            probs = F.softmax(logits, dim=-1)
            next_idx = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_idx), dim=1)

            # Early stopping if stop_token is generated
            if stop_token:
                stop_token_id = encode(stop_token)[0]
                if next_idx.item() == stop_token_id:
                    break
        return idx


In [8]:
model = LanguageModel().to(device)
model.load_state_dict(torch.load("clash_transformer_finetuned.pth"))

C:\Users\yuges\AppData\Local\Temp\ipykernel_10712\128376389.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("clash_transformer_finetuned

<All keys matched successfully>

In [15]:
def count_parameters_in_millions(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total parameters: {total / 1_000_000:.2f}M")
    print(f"Trainable parameters: {trainable / 1_000_000:.2f}M")


count_parameters_in_millions(model)


Total parameters: 2.96M
Trainable parameters: 2.96M


In [ ]:

with torch.no_grad():
    # prompt = "<|user|> Describe the Royal Champion's ability <|assistant|>"
    while True:
         user_input =  input("User Prompt: ").strip()
         if user_input.lower() == "exit":
                print("Exiting the chat. Goodbye!")
                break
         if user_input:
            prompt = f"<|user|> {user_input} <|assistant|>"
            
            input_ids = torch.tensor([encode(prompt)], dtype=torch.long).to(device)
            output_ids = model.generate(
                input_ids,
                max_new_tokens=400,
                temperature=0.8,
                top_k=40,
                stop_token="<|endoftext|>"
            )

            # Decode
            full_text = decode(output_ids[0].tolist())

            # Extract only assistant reply
            start = full_text.find("<|assistant|>") + len("<|assistant|>")
            end = full_text.find("<|endoftext|>", start)
            assistant_response = full_text[start:end].strip() if end != -1 else full_text[start:].strip()

            print("\n🧠 Assistant Response:\n")
            print(assistant_response)


🧠 Assistant Response:

The Royal Champion is unlocked at Town Hall 13 and is designed to target and destroy defensive structures. Her ability, Seeking Shield, throws a shield that hits up to four defenses, dealing high damage. She can jump over walls, making her highly effective at taking out key defenses like Inferno Towers and Eagle Artillery. <
